In [1]:
import pandas as pd

## Ingest Data

In [2]:
raw_recipes_df = pd.read_csv("../data/RAW_recipes.csv")#, index_col="id"

In [3]:
recipe_df = raw_recipes_df[["id", "ingredients"]]
recipe_df.head()

,id,ingredients
0,137739,"['winter squash', 'mexican seasoning', 'mixed ..."
1,31490,"['prepared pizza crust', 'sausage patty', 'egg..."
2,112140,"['ground beef', 'yellow onions', 'diced tomato..."
3,59389,"['spreadable cheese with garlic and herbs', 'n..."
4,44061,"['tomato juice', 'apple cider vinegar', 'sugar..."


In [4]:
recipe_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231637 entries, 0 to 231636
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   id           231637 non-null  int64 
 1   ingredients  231637 non-null  object
dtypes: int64(1), object(1)
memory usage: 3.5+ MB


## Data Cleaning
### Explode the list of ingredients per recipe

In [5]:
from ast import literal_eval

# Convert to list so explode works
recipe_df["ingredients"] = recipe_df["ingredients"].apply(literal_eval)

ingredients_exploded_df = recipe_df.explode("ingredients", ignore_index=True)

/tmp/ipykernel_126/446667703.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recipe_df["ingredients"] = recipe_df["ingredients"].apply(literal_eval)


In [6]:
display(ingredients_exploded_df)

,id,ingredients
0,137739,winter squash
1,137739,mexican seasoning
2,137739,mixed spice
3,137739,honey
4,137739,butter
...,...,...
2096577,298509,eggs
2096578,298509,flour
2096579,298509,cream of tartar
2096580,298509,baking soda


In [7]:
ingredients_exploded_df.groupby("ingredients").count()

,id
ingredients,
1% fat buttermilk,24
1% fat cottage cheese,32
1% low-fat chocolate milk,2
1% low-fat milk,472
10 bean soup mix,2
...,...
zoom quick hot cereal,1
zucchini,4591
zucchini with italian-style tomato sauce,3


### Group by ingredient to get a feel for the least common

This helps to identify areas that perhaps could use some cleaning, such as removing brand names

In [8]:
ingredient_counts_df = ingredients_exploded_df.groupby("ingredients").count().rename(columns={"id": "count"})

Reset the index so we can split the data even more

In [9]:
ingredient_counts_df = ingredient_counts_df.reset_index()

In [10]:
ingredient_counts_df.sort_values("count", ascending=True).head()

,ingredients,count
14941,zwieback toast crumbs,1
5379,frozen cranberry-apple juice cocktail,1
5374,frozen corn souffle,1
12562,soyaki,1
5360,frozen chopped mangoes,1


Split the ingredient values so that we can easily remove words that are less necessary to the integrity of the recipe

In [11]:
ingredient_words = ingredient_counts_df.ingredients.str.split().rename("words")

In [12]:
ingredient_df = pd.concat([ingredient_counts_df, ingredient_words], axis=1)

In [13]:
display(ingredient_df)

,ingredients,count,words
0,1% fat buttermilk,24,"[1%, fat, buttermilk]"
1,1% fat cottage cheese,32,"[1%, fat, cottage, cheese]"
2,1% low-fat chocolate milk,2,"[1%, low-fat, chocolate, milk]"
3,1% low-fat milk,472,"[1%, low-fat, milk]"
4,10 bean soup mix,2,"[10, bean, soup, mix]"
...,...,...,...
14937,zoom quick hot cereal,1,"[zoom, quick, hot, cereal]"
14938,zucchini,4591,[zucchini]
14939,zucchini with italian-style tomato sauce,3,"[zucchini, with, italian-style, tomato, sauce]"
14940,zwieback toast,5,"[zwieback, toast]"


### Explore most commonly occurring words

In [14]:
all_words = ingredient_df.explode("words", ignore_index=True)

In [15]:
display(all_words.groupby("words").count().sort_values("count", ascending=False)[:10])

,ingredients,count
words,,
mix,649,649
cheese,562,562
sauce,449,449
chocolate,445,445
and,361,361
chicken,356,356
dried,327,327
cream,323,323
red,305,305


In [16]:
all_words[all_words["ingredients"].str.contains("cheese")]

,ingredients,count,words
3,1% fat cottage cheese,32,1%
4,1% fat cottage cheese,32,fat
5,1% fat cottage cheese,32,cottage
6,1% fat cottage cheese,32,cheese
122,2% cheddar cheese,68,2%
...,...,...,...
40786,whole milk ricotta cheese,95,cheese
41377,yellow cheese,18,yellow
41378,yellow cheese,18,cheese
41528,yogurt cheese,19,yogurt


### Remove descriptive words

These words complicate the data. Essentially, a 9in tortilla is the same as an 8in tortilla. low-fat, low-carb, and reduced fat don"t have a strong influence on the dish. Therefore, I drop them.

In [17]:
df_merge_exploded = ingredient_df.explode("words")

adjectives_to_remove = ["kraft", "betty", "crocker", "duncan", "hines",\
                        "low-fat", "reduced", "fat", "fat-free", "low-carb", "reduced-fat",\
                        "prepared", "fresh", "frozen", "grated", "unsweetened",\
                        "free", "sugar-free", "and", "salad", "wishbone",\
                        "whole", "dried", "inch", "in.", "baby", "cooked", "extra",\
                       "large", "extra-large", "dry", "best", "unsalted", "vegan", "vegetarian", \
                       "whole", "wheat", "toasted", "unbaked", "unbleached", "unsifted", "boneless", "breaded", "canned",\
                       "lean", "half", "skinless", "100-calorie", "%", "10-minute", "bottled", "brewed", "de-fatted", "boneless"] 
adjectives_to_remove.extend([f"{i}%" for i in range(0,101)])
adjectives_to_remove.extend(["8-in", "8\"", "9-in", "9\"", "10\""])
adjectives_to_remove.extend([f"{i}-inch" for i in range(1,24)])
adjectives_to_remove.extend([str(i) for i in range(0, 100)])
adjectives_to_remove.extend(["red", "yellow", "green", "brown"])

simplified_ingredient_list_df = df_merge_exploded[~df_merge_exploded["words"].isin(adjectives_to_remove)]

In [18]:
display(simplified_ingredient_list_df.groupby("words").count().sort_values("count", ascending=False)[:25])

,ingredients,count
words,,
mix,649,649
cheese,562,562
sauce,449,449
chocolate,445,445
chicken,356,356
cream,323,323
beef,284,284
white,265,265
juice,250,250


In [19]:
simplified_ingredient_list_df.head()

,ingredients,count,words
0,1% fat buttermilk,24,buttermilk
1,1% fat cottage cheese,32,cottage
1,1% fat cottage cheese,32,cheese
2,1% low-fat chocolate milk,2,chocolate
2,1% low-fat chocolate milk,2,milk


In [20]:
import inflection as inf

def combine_with_stop_words(x):
    arr = []
    for item in x:
        if item == "with":
            break
        if "\"" in item:
            continue
            
        singularized = inf.singularize(item)
        
        # Misspelling in singularize
        misspellings = {
            "cooky": "cookie",
            "potatoe": "potato"
        }
        if singularized in misspellings:
            singularized = misspellings[singularized]
            
            
        arr.append(singularized)
            
    return " ".join(arr)

In [21]:
ingredient_df["new_name"] = simplified_ingredient_list_df.groupby(level=0, axis=0)["words"].agg(lambda x: combine_with_stop_words(x))

In [22]:
df_with_new_counts = ingredient_df.join(ingredient_df.groupby("new_name").sum(numeric_only=True).rename(columns={"count": "new_count"}), on="new_name")

In [23]:
df_with_new_counts.sort_values("new_count", ascending=True)[:25]

,ingredients,count,words,new_name,new_count
14941,zwieback toast crumbs,1,"[zwieback, toast, crumbs]",zwieback toast crumb,1.0
3700,dark chocolate-covered cranberries,1,"[dark, chocolate-covered, cranberries]",dark chocolate-covered cranberry,1.0
10927,raspberry-cranberry preserves,1,"[raspberry-cranberry, preserves]",raspberry-cranberry preserf,1.0
10925,raspberry zinger tea bags,1,"[raspberry, zinger, tea, bags]",raspberry zinger tea bag,1.0
10923,raspberry white wine vinegar,1,"[raspberry, white, wine, vinegar]",raspberry white wine vinegar,1.0
3707,dark mexican beer,1,"[dark, mexican, beer]",dark mexican beer,1.0
10916,raspberry seltzer water,1,"[raspberry, seltzer, water]",raspberry seltzer water,1.0
10912,raspberry red wine vinegar,1,"[raspberry, red, wine, vinegar]",raspberry wine vinegar,1.0
10908,raspberry pastry filling,1,"[raspberry, pastry, filling]",raspberry pastry filling,1.0
10907,raspberry muffin mix,1,"[raspberry, muffin, mix]",raspberry muffin mix,1.0


## Drop the least repeated ingredients
This is something I'd like to circle back on, but for now let's ignore any ingredeint not used at least 10 times.

In [24]:
df_with_new_counts.loc[df_with_new_counts["new_count"] <= 10 , "new_name"] = "Other"

In [25]:
print(f"original ingredient count {pd.unique(df_with_new_counts.ingredients).shape[0]}")

original ingredient count 14942


In [26]:
print(f"new ingredient count {pd.unique(df_with_new_counts.new_name).shape[0]}")

new ingredient count 4297


In [27]:
pd.unique(df_with_new_counts.new_name)[1100:1200]

array(['condensed cream of chicken soup',
       'condensed cream of mushroom & garlic soup',
       'condensed cream of mushroom soup',
       'condensed cream of potato soup', 'condensed french onion soup',
       'condensed golden mushroom soup', 'condensed milk',
       'condensed tomato soup', 'condiment', "confectioners' sugar",
       'converted long grain rice', 'converted rice',
       'converted white rice', 'corkscrew macaroni', 'corned beef',
       'couscou', 'dark chicken meat', 'ditalini', 'duck', 'egg noodle',
       'elbow macaroni', 'farfalle pastum', 'fettuccine', 'glutinou rice',
       'grit', 'ham shank', 'italian sausage', 'jasmine rice',
       'lasagna noodle', 'ground turkey', 'linguine', 'lobster meat',
       'long-grain rice', 'macaroni', 'minute rice', 'noodle', 'oatmeal',
       'oat', 'orzo pastum', 'pastum', 'pastum shell', 'pearl barley',
       'penne', 'pot roast', 'quinoa', 'radiatore', 'rigatoni pastum',
       'rotini', 'sausage', 'scallop', 'shor

## One-hot encode
Now that we have a simplified ingredient list, explode the list to columns then join back on the original recipe list

In [28]:
one_hot_encoded_ingredients = pd.get_dummies(df_with_new_counts["new_name"], prefix="ingredient")

In [29]:
ingredient_df_encoded = pd.concat([df_with_new_counts, one_hot_encoded_ingredients], axis = 1)
ingredient_df_encoded = ingredient_df_encoded.set_index("ingredients")

In [30]:
0 // 0

ZeroDivisionError: integer division or modulo by zero

In [ ]:
df_recipes_one_hot = ingredients_exploded_df.join(ingredient_df_encoded, on="ingredients")

In [ ]:
# df_recipes_one_hot.to_pickle("tmp_df.pkl")

In [ ]:
# df_recipes_one_hot = pd.read_pickle("tmp_df.pkl")

In [ ]:
df_recipes_one_hot.drop(["ingredients", "count", "words", "new_name", "new_count"], axis=1, inplace=True)

In [ ]:
recipe_encoded_df = df_recipes_one_hot.groupby("id").max()

In [ ]:
recipe_encoded_df.drop(["ingredients", "count", "words", "new_count"], axis=1, inplace=True)

In [ ]:
# recipe_encoded_df.to_pickle("../cleaned_data/recipe_encoded_df.pkl")

In [33]:
recipe_encoded_df

,ingredient_,ingredient_& light cream,ingredient_7-up,ingredient_7-up soda,ingredient_Other,ingredient_a.1. original sauce,ingredient_absolut citron vodka,ingredient_absolut mandarin vodka,ingredient_absolut vodka,ingredient_accent seasoning,...,ingredient_yoghurt,ingredient_yogurt,ingredient_yogurt cheese,ingredient_your favorite barbecue rub,ingredient_yucca root,ingredient_yukon gold potato,ingredient_zinfandel,ingredient_ziploc bag,ingredient_ziti pastum,ingredient_zucchini
id,,,,,,,,,,,,,,,,,,,,,
38,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
43,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537459,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537485,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537543,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
